In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import confusion_matrix

In [2]:
def splitDataset(dataset, trainRatio):
    trainSize = int(len(dataset) * trainRatio)
    trainSet = []
    copy = list(dataset.iloc[:,:].values)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [np.array(trainSet), np.array(copy)]

def euc_distance(lista, x):
    if(len(lista) == len(x)):
        n=len(lista)-1
    else:
        return('Os conjuntos devem possuir a mesma dimensão!')
    dist = 0
    for i in range(n):
        dist += (lista[i]-x[i])**2
    return(np.sqrt(dist))

def calc_distances(train_set, amostra):
    distances=[]
    for i in range(len(train_set)):
        dist = euc_distance(train_set[i],amostra)
        distances.append((dist,train_set[i][-1]))
    return(distances)

def att_class(dataset,amostra):
    distances=calc_distances(dataset,amostra)
    distances.sort()
    return(distances[0][1])

def make_pred(dataset,amostras):
    preds=[]
    for amostra in amostras:
        preds.append(att_class(dataset, amostra))
    return(preds)

In [3]:
def prepare_to_remove(dataset):
    """
    Calcula algumas estatísticas para a remoção de outliers do dataset.
    
    Input
    ----------
    dataset: 
    Dataset a ser preparado para remoção de outliers.
    
    Output
    ----------
    cols:
    Lista contendo os nomes das colunas dos datasets.
  
    means:
    Lista contendo as médias das colunas dos datasets.
  
    stds:
    Lista contendo os desios padrões das colunas dos datasets.
  
    """
    cols, means, stds = [],[],[]
    for col in dataset.columns:
        try:
            cols.append(col)
            means.append(dataset[col].mean())
            stds.append(dataset[col].std())
        except TypeError:
            means.append(np.nan)
            stds.append(np.nan)
            print(f'Coluna {col} possui valores em formato não númerico!')
    return(cols, means, stds)

def remove_outliers(dataset):
    """
    Soma todos os elementos de um vetor, exceto o que está na posição n.
    
    Input
    ----------
    dataset: 
    Dataset para remoção de outliers.
    
    Output
    ----------
    dataset:
    O dataset da entrada, agora com os outliers removidos
    
    
    Obs.: São consideradas outliers da variável x, observações além do intervalo: mean(x) ± 2*std(x).
    """
    cols, means, stds = prepare_to_remove(dataset)
    k=0
    for col in cols:
        dataset = dataset.loc[(dataset[col] > means[k]-2*stds[k]) & (dataset[col] < means[k]+2*stds[k])]
        k=k+1
    return(dataset)


In [4]:
data=pd.read_csv('https://raw.githubusercontent.com/rhanielmx/RecPad/master/messidor_features.csv')
data=remove_outliers(data)
feature_importances=[]
cols_names=[]
for i in range(50):
    train_set, test_set = splitDataset(dataset=data,trainRatio=0.7)
    X_train,y_train=[train_set[i][:-1] for i in range(train_set.shape[0])],[train_set[i][-1] for i in range(train_set.shape[0])]
    X_test,y_test=[test_set[i][:-1] for i in range(test_set.shape[0])],[test_set[i][-1] for i in range(test_set.shape[0])]

    from sklearn.ensemble import RandomForestClassifier 
    rf = RandomForestClassifier() 
    rf.fit(X_train, y_train) 

    feature_importances.append(rf.feature_importances_)

avg_feature_importances=sum(feature_importances)/len(feature_importances)
avg_feature_importances=pd.DataFrame(avg_feature_importances*100,columns=['Average Importance(%)'],index=list(data.columns[:-1]))

print(avg_feature_importances.sort_values('Average Importance(%)',ascending=False))
cols_to_use=[avg_feature_importances.index[i] for i in range(len(avg_feature_importances.index)) if (avg_feature_importances.iloc[i,0]>=(avg_feature_importances).mean()[0])==True]
cols_to_use.append('Target')

                            Average Importance(%)
Exudates_Detection_1                     9.094294
MA_Detection_alpha-0.5                   8.167433
Exudates_Detection_2                     7.872369
Macula_OpticDisc_Distance                7.805295
OpticDisc_Diameter                       7.754035
Exudates_Detection_3                     7.189910
Exudates_Detection_4                     6.967984
MA_Detection_alpha-0.6                   6.085020
MA_Detection_alpha-0.7                   5.708789
MA_Detection_alpha-1.0                   5.545903
MA_Detection_alpha-0.9                   5.472036
MA_Detection_alpha-0.8                   5.242609
Exudates_Detection_5                     4.772039
Exudates_Detection_7                     4.743409
Exudates_Detection_6                     3.647565
Exudates_Detection_8                     3.007950
AM/FM-based classification               0.923362
Pre-Screening                            0.000000
Quality_Assessment                       0.000000


In [5]:
def main():    
    for _ in range(n_rounds):
        data=pd.read_csv(filepath_or_buffer=path,usecols=cols_to_use)
        data=remove_outliers(data)
        train_set, test_set = splitDataset(dataset=data,trainRatio=trainRatio)
        y_test=[test_set[i][-1] for i in range(test_set.shape[0])]
        
        preds=make_pred(train_set, test_set)
        
        
        cm = confusion_matrix(y_test, preds)
        sr = 100*(cm.diagonal().sum()/cm.sum())
        confusion_matrixes.append(cm)
        accuracys.append(sr)
        
        class0_success=100*cm[0][0]/(cm[0][0]+cm[0][1])
        class1_success=100*cm[1][1]/(cm[1][0]+cm[1][1])
         
        class0_successes.append(class0_success)
        class1_successes.append(class1_success)
        
        if ((_+1)%(int(n_rounds/10))==0):
            print(f'{(_+1)*(100/n_rounds):05.2f}% concluído!')

In [6]:
accuracys = []
confusion_matrixes = []
class0_successes,class1_successes=[],[]
trainRatio=0.7
n_rounds=100
path='https://raw.githubusercontent.com/rhanielmx/RecPad/master/messidor_features.csv'

%time main()

10.00% concluído!
20.00% concluído!
30.00% concluído!
40.00% concluído!
50.00% concluído!
60.00% concluído!
70.00% concluído!
80.00% concluído!
90.00% concluído!
100.00% concluído!
Wall time: 5min 37s


In [7]:
Accuracys=pd.DataFrame(data={'Accuracys':accuracys})
Accuracys.describe()

,Accuracys
count,100.000000
mean,60.184783
std,2.647954
min,54.347826
25%,58.333333
50%,60.326087
75%,62.047101
max,65.942029


In [8]:
Class_Sucesses=pd.DataFrame(data={'Succces 0':class0_successes,'Succces 1':class1_successes})
Class_Sucesses.describe()

,Succces 0,Succces 1
count,100.000000,100.000000
mean,61.811406,58.588029
std,3.849646,4.250281
min,54.140127,49.659864
25%,59.321429,55.823055
50%,61.444077,58.866925
75%,64.613971,61.363636
max,70.370370,69.421488
